## 大致統整有4種適用的模型用於預測數據
1. 一對多 2. 一對一 3. 多對一 4.多對多

一對一
model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))  
model.add(TimeDistributed(Dense(1)))    #可以建這層   or model.add(Dense(1))  
  
一對多  
model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))  
model.add(Dense(1))  
model.add(RepeatVector(5)) # 增加維度 if input(None,32) output(None,5,32)  
  
多對一  
model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))  
model.add(Dense(1))  
  
多對多  
model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))  
model.add(TimeDistributed(Dense(1)))

In [10]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

# =========================================匯入資料並處理=========================================

def readTrain():
    train = pd.read_csv(r'D:\AYA\test_file\SPY.csv')
    return train

def augFeatures(train):
    train["Date"] = pd.to_datetime(train["Date"])
    train["year"] = train["Date"].dt.year
    train["month"] = train["Date"].dt.month
    train["date"] = train["Date"].dt.day
    train["day"] = train["Date"].dt.dayofweek
    return train

# 所有數據正規化 (月份、日期也可以)
def normalize(train):
    train = train.drop(["Date"], axis=1)
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm


# 輸入X_train: 利用前30天的Open, High, Low, Close, Adj Close, 
# Volume, month, year, date, day作為Features，shape為(30, 10)
# Y_train: 利用未來5天的Adj Close作為Features，shape為(5,1)

def buildTrain(train, pastDay, futureDay):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["Adj Close"]))
    return np.array(X_train), np.array(Y_train)

# X_train,Y_train = buildTrain(train, 1, 1)

# 將資料打散，而非照日期排序
def shuffle(X,Y):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

# X,Y = shuffle(X_train,Y_train)

# 將Training Data取一部份當作Validation Data
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

# X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [11]:
# =========================================建模型=========================================

# 一對一的模型 因此return_sequences 也可設為False ，
# 但Y_train 以及Y_val的維度需改為二維(5710,1)以及(634,1)
def buildOneToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
    # output shape: (1, 1)
    model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1)) # 多對一的模型不能建這層
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

# 將過去的天數pastDay設為1，預測的天數futureDay也設為1

train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]

model = buildOneToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


Instructions for updating:
Colocations handled automatically by placer.


C:\Users\arthur.chang\AppData\Local\Continuum\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  import sys
C:\Users\arthur.chang\AppData\Local\Continuum\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(1, 10))`
  import sys


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 10)             840       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 5710 samples, validate on 634 samples
Epoch 1/1000
5710/5710 [==============================] - 1s 147us/step - loss: 0.0719 - val_loss: 0.0531
Epoch 2/1000
5710/5710 [==============================] - 0s 16us/step - loss: 0.0424 - val_loss: 0.0289
Epoch 3/1000
5710/5710 [==============================] - 0s 15us/step - loss: 0.0203 - val_loss: 0.0114
Epoch 4/1000
5710/5710 [==============================] - 0s 15us/step - loss: 0.0065 - val_loss: 0.0027
Ep

Epoch 65/1000
5710/5710 [==============================] - 0s 11us/step - loss: 3.7907e-05 - val_loss: 4.0310e-05
Epoch 66/1000
5710/5710 [==============================] - 0s 13us/step - loss: 3.7818e-05 - val_loss: 3.9704e-05
Epoch 67/1000
5710/5710 [==============================] - 0s 13us/step - loss: 3.7455e-05 - val_loss: 3.9387e-05
Epoch 68/1000
5710/5710 [==============================] - 0s 13us/step - loss: 3.7098e-05 - val_loss: 3.9526e-05
Epoch 69/1000
5710/5710 [==============================] - 0s 13us/step - loss: 3.6917e-05 - val_loss: 3.8501e-05
Epoch 70/1000
5710/5710 [==============================] - 0s 13us/step - loss: 3.6594e-05 - val_loss: 3.8037e-05
Epoch 71/1000
5710/5710 [==============================] - 0s 13us/step - loss: 3.6166e-05 - val_loss: 3.8876e-05
Epoch 72/1000
5710/5710 [==============================] - 0s 13us/step - loss: 3.6150e-05 - val_loss: 3.7583e-05
Epoch 73/1000
5710/5710 [==============================] - 0s 13us/step - loss: 3.5710e-

5710/5710 [==============================] - 0s 13us/step - loss: 2.4263e-05 - val_loss: 2.3836e-05
Epoch 137/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.4627e-05 - val_loss: 2.3329e-05
Epoch 138/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.4472e-05 - val_loss: 2.3679e-05
Epoch 139/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.4230e-05 - val_loss: 2.3727e-05
Epoch 140/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.4269e-05 - val_loss: 2.3802e-05
Epoch 141/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.4377e-05 - val_loss: 2.3633e-05
Epoch 142/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.4293e-05 - val_loss: 2.3907e-05
Epoch 143/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.3893e-05 - val_loss: 2.3486e-05
Epoch 144/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.3791e-05 - v

In [4]:
# 多對一模型
# LSTM參數return_sequences=False，未設定時default也為False，
# 而且不可使用TimeDistribution

def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
    # output shape: (1, 1)
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model
# 需要設定的有pastDay=30、future=1 ，且注意Y_train 的維度需為二維
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
# https://medium.com/ai%E5%8F%8D%E6%96%97%E5%9F%8E/learning-model-earlystopping%E4%BB%8B%E7%B4%B9-%E8%BD%89%E9%8C%84-f364f4f220fb
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


C:\Users\arthur.chang\AppData\Local\Continuum\anaconda2\envs\Python3.6\lib\site-packages\ipykernel\__main__.py:7: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\arthur.chang\AppData\Local\Continuum\anaconda2\envs\Python3.6\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(30, 10))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5684 samples, validate on 631 samples
Epoch 1/1000
5684/5684 [==============================] - 2s 288us/step - loss: 0.1267 - val_loss: 0.0494
Epoch 2/1000
5684/5684 [==============================] - 1s 91us/step - loss: 0.0200 - val_loss: 0.0066
Epoch 3/1000
5684/5684 [==============================] - 0s 87us/step - loss: 0.0031 - val_loss: 0.0015
Epoch 4/1000
5684/5684 [==============================] - 1s 88us/step - loss: 0.0011 - val_loss: 8.6022e-04
Epoch 5/1000
5684/5684 [==============================] - 0s 88us/st

5684/5684 [==============================] - 1s 106us/step - loss: 2.9567e-05 - val_loss: 3.1751e-05
Epoch 139/1000
5684/5684 [==============================] - 1s 105us/step - loss: 2.8920e-05 - val_loss: 3.6001e-05
Epoch 140/1000
5684/5684 [==============================] - 1s 105us/step - loss: 2.8463e-05 - val_loss: 3.4419e-05
Epoch 141/1000
5684/5684 [==============================] - 1s 111us/step - loss: 2.8349e-05 - val_loss: 3.0679e-05
Epoch 142/1000
5684/5684 [==============================] - 1s 105us/step - loss: 2.9035e-05 - val_loss: 3.2463e-05
Epoch 143/1000
5684/5684 [==============================] - 1s 104us/step - loss: 2.8574e-05 - val_loss: 3.0498e-05
Epoch 144/1000
5684/5684 [==============================] - 1s 108us/step - loss: 2.8515e-05 - val_loss: 3.4697e-05
Epoch 145/1000
5684/5684 [==============================] - 1s 106us/step - loss: 2.8572e-05 - val_loss: 3.0497e-05
Epoch 146/1000
5684/5684 [==============================] - 1s 106us/step - loss: 2.834

In [6]:
# 一對多模型
# 因為是一對多模型Timesteps只有1，因此return_sequences=False 才可執行

def buildOneToManyModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
    # output shape: (5, 1)
    model.add(Dense(1))
    model.add(RepeatVector(5)) # 增加維度 if input(None,32) output(None,5,32)
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

# 將pastDay 設為1, futureDay 設為5

train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildOneToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

C:\Users\arthur.chang\AppData\Local\Continuum\anaconda2\envs\Python3.6\lib\site-packages\ipykernel\__main__.py:6: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\arthur.chang\AppData\Local\Continuum\anaconda2\envs\Python3.6\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 10))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 1)              0         
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5706 samples, validate on 634 samples
Epoch 1/1000
5706/5706 [==============================] - 1s 189us/step - loss: 0.0716 - val_loss: 0.0550
Epoch 2/1000
5706/5706 [==============================] - 0s 14us/step - loss: 0.0420 - val_loss: 0.0293
Epoch 3/1000
5706/5706 [==============================] - 0s 11us/step - loss: 0.0200 - val_loss: 0.0113
Epoch 4/1000
5706/5706 [=====================

5706/5706 [==============================] - 0s 11us/step - loss: 5.7333e-05 - val_loss: 5.4981e-05
Epoch 138/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7601e-05 - val_loss: 5.7041e-05
Epoch 139/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7830e-05 - val_loss: 5.5369e-05
Epoch 140/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7573e-05 - val_loss: 5.5895e-05
Epoch 141/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7298e-05 - val_loss: 5.6630e-05
Epoch 142/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7589e-05 - val_loss: 5.5459e-05
Epoch 143/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7095e-05 - val_loss: 5.6545e-05
Epoch 144/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7297e-05 - val_loss: 5.5431e-05
Epoch 145/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7265e-05 - v

In [13]:
Y_val.ndim

3

In [11]:
X_train.shape

(5706, 1, 10)

In [9]:
X_train[0]

array([[ 0.14899517,  0.14694595,  0.14490678,  0.1411666 ,  0.15787337,
         0.14618045,  0.31321147, -0.04660058, -0.35728018, -0.00508194]])

In [12]:
# 多對多模型(輸入與輸出不同長度)
# 將return_sequences 設為True ，再用TimeDistributed(Dense(1)) 將輸出調整為(5,1)
def buildManyToManyModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
    # output shape: (5, 1)
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 5, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildManyToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


C:\Users\arthur.chang\AppData\Local\Continuum\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\arthur.chang\AppData\Local\Continuum\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(5, 10))`
  """


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 5, 10)             840       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5703 samples, validate on 633 samples
Epoch 1/1000
5703/5703 [==============================] - 1s 171us/step - loss: 0.0996 - val_loss: 0.0552
Epoch 2/1000
5703/5703 [==============================] - 0s 33us/step - loss: 0.0360 - val_loss: 0.0170
Epoch 3/1000
5703/5703 [==============================] - 0s 31us/step - loss: 0.0125 - val_loss: 0.0087
Epoch 4/1000
5703/5703 [==============================] - 0s 31us/step - loss: 0.0081 - val_loss: 0.0067
Epoch 5/1000
5703/5703 [==========================

5703/5703 [==============================] - 0s 29us/step - loss: 1.0273e-04 - val_loss: 1.0650e-04
Epoch 69/1000
5703/5703 [==============================] - 0s 29us/step - loss: 1.0097e-04 - val_loss: 1.0766e-04
Epoch 70/1000
5703/5703 [==============================] - 0s 30us/step - loss: 1.0072e-04 - val_loss: 1.0642e-04
Epoch 71/1000
5703/5703 [==============================] - 0s 29us/step - loss: 1.0009e-04 - val_loss: 1.0518e-04
Epoch 72/1000
5703/5703 [==============================] - 0s 29us/step - loss: 9.9820e-05 - val_loss: 1.0404e-04
Epoch 73/1000
5703/5703 [==============================] - 0s 29us/step - loss: 9.9366e-05 - val_loss: 1.0541e-04
Epoch 74/1000
5703/5703 [==============================] - 0s 28us/step - loss: 9.8850e-05 - val_loss: 1.0458e-04
Epoch 75/1000
5703/5703 [==============================] - 0s 29us/step - loss: 9.8258e-05 - val_loss: 1.0311e-04
Epoch 76/1000
5703/5703 [==============================] - 0s 30us/step - loss: 9.7876e-05 - val_loss:

5703/5703 [==============================] - 0s 31us/step - loss: 8.8571e-05 - val_loss: 9.5522e-05
Epoch 140/1000
5703/5703 [==============================] - 0s 30us/step - loss: 8.8952e-05 - val_loss: 9.7100e-05
Epoch 141/1000
5703/5703 [==============================] - 0s 27us/step - loss: 8.7942e-05 - val_loss: 9.5892e-05
Epoch 142/1000
5703/5703 [==============================] - 0s 33us/step - loss: 8.8671e-05 - val_loss: 9.9365e-05
Epoch 143/1000
5703/5703 [==============================] - 0s 29us/step - loss: 8.7905e-05 - val_loss: 9.6489e-05
Epoch 144/1000
5703/5703 [==============================] - 0s 31us/step - loss: 8.9507e-05 - val_loss: 9.8852e-05
Epoch 145/1000
5703/5703 [==============================] - 0s 29us/step - loss: 8.8755e-05 - val_loss: 9.5407e-05
Epoch 146/1000
5703/5703 [==============================] - 0s 28us/step - loss: 8.8501e-05 - val_loss: 9.5517e-05
Epoch 147/1000
5703/5703 [==============================] - 0s 31us/step - loss: 8.9818e-05 - v